In [1]:
# imports
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
import itertools

## Functions

In [2]:
# functions
def num_age(x):
    '''make age categories numeric'''
    if x == '18-39':
        return 0
    elif x == '40-44':
        return 1
    elif x == '45-49':
        return 2
    elif x == '50-54':
        return 3
    elif x == '55-59':
        return 4
    elif x == '60-64':
        return 5
    elif x == '65-69':
        return 6
    return 7


def cat_clean(x):
    '''binary columns were floats, make them ints'''
    return int(x)


def prep(df):
    '''prepare dataframe for EDA'''
    df['age'] = df['age'].apply(lambda x: num_age(x))
    df['clear'] = df['clear'].apply(lambda x: cat_clean(x))
    df['overcast'] = df['overcast'].apply(lambda x: cat_clean(x))
    df['partially_cloudy'] = df['partially_cloudy'].apply(lambda x: cat_clean(x))
    df['rain'] = df['rain'].apply(lambda x: cat_clean(x))

    cols = ['year', 'age', 'male', 'time_seconds', 'minimum_temperature',
           'maximum_temperature', 'temperature', 'relative_humidity', 'wind_speed',
           'precipitation', 'precipitation_cover', 'cloud_cover', 'clear',
           'overcast', 'partially_cloudy', 'rain']

    for col in cols:
        df[col] = pd.to_numeric(df[col], downcast='unsigned')

    df.drop(columns=['maximum_temperature', 'minimum_temperature'], inplace=True)

    event_dict = {year: df[df['year'] == year] for year in df['year'].unique()}
    
    return event_dict


def usable(event_dict, num):
    '''specify which years are usable based on minimum number of participants
    returns dict with years that exceed minimum values are correspoding df'''
    can_use = []
    for year in event_dict.keys():
        if len(event_dict[year]) > num:
            can_use.append(year)
    return {year: event_dict[year] for year in can_use}


def top_n(event_dict, num):
    '''take in event dict and first num of finishers that are desired,
    return df of top n racers for all years, use in conjuction with usable()'''
    df = pd.DataFrame()
    for year in event_dict.keys():
        df = pd.concat([df, event_dict[year].sort_values(by='time_seconds').iloc[:num]], ignore_index=True)
    return df
        
    
def n_range(event_dict, low, high):
    '''take in dictionary of dfs from prep(), upper percentage as float, lower percentage as float
    return df'''
    df = pd.DataFrame()
    for year in event_dict.keys():
        df = pd.concat([df, event_dict[year].sort_values(by='time_seconds').iloc[int(len(event_dict[year]) * low): int(len(event_dict[year]) * high)]], ignore_index=True)
    return df

def model_scores(X_test, y_test, model):
    nl = "\n"
    y_preds = model.predict(X_test)
    return print(f'R2: {model.score(X_test, y_test)}{nl}MAE: {mean_absolute_error(y_test, y_preds)}{nl}MSE: {mean_squared_error(y_test, y_preds)}{nl}RMSE: {mean_squared_error(y_test, y_preds, squared=False)}')

## Modeling Ridge

In [3]:
london = pd.read_csv('./data/London_Data/Clean/Clean_London_Results_Weather.csv')
nyc = pd.read_csv('./data/NYC_Data/Clean/Clean_NYC_Results_Weather.csv')
boston = pd.read_csv('./data/Boston_Data/Clean/Clean_Boston_Results_Weather.csv')
berlin = pd.read_csv('./data/Berlin_Data/Clean/Clean_Berlin_Results_Weather.csv')
chicago = pd.read_csv('./data/Chicago_Data/Clean/Clean_Chicago_Results_Weather.csv')

london_dict = prep(london)
nyc_dict = prep(nyc)
boston_dict = prep(boston)
berlin_dict = prep(berlin)
chicago_dict = prep(chicago)

london = n_range(usable(london_dict, 9_900), 0, 1)
nyc = n_range(usable(nyc_dict, 9_900), 0, 1)
boston = n_range(usable(boston_dict, 9_900), 0, 1)
berlin = n_range(usable(berlin_dict, 9_900), 0, 1)
chicago = n_range(usable(chicago_dict, 9_900), 0, 1)

aggregate = {col: 'mean' for col in london.drop(columns=['year'])}

london = london.groupby(['year', 'age', 'male']).agg(aggregate)
nyc = nyc.groupby(['year', 'age', 'male']).agg(aggregate)
boston = boston.groupby(['year', 'age', 'male']).agg(aggregate)
berlin = berlin.groupby(['year', 'age', 'male']).agg(aggregate)
chicago = chicago.groupby(['year', 'age', 'male']).agg(aggregate)

events = [boston, berlin, chicago, london, nyc]
count = 0
for event in events:
    event['event'] = count
    count += 1
    
combined = pd.concat(events)
combined = combined.merge(pd.get_dummies(combined['event'], drop_first=True), left_index=True, right_index=True).drop(columns='event')
combined.columns = ['age', 'male','time_seconds', 'temperature', 'relative_humidity','wind_speed','precipitation',
                    'precipitation_cover', 'cloud_cover', 'clear', 'overcast', 'partially_cloudy', 'rain', 'berlin', 'chicago', 'london', 'nyc']

In [53]:
rparams = {
    'alpha': [.01, .1, 1, 10, 100, 1000],
    'tol': [1, .1, .01, .001, .0001, .00001],
}

rgs = GridSearchCV(
    estimator=Ridge(),
    param_grid=rparams
)

In [54]:
train = london[london.index.isin(range(2017), level=0)]
test = london[london.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


rgs.fit(X_train, y_train)
model_scores(X_test, y_test, rgs)

R2: 0.8504044279163465
MAE: 504.9274927454586
MSE: 359577.15313391085
RMSE: 599.6475240788632


In [55]:
pd.DataFrame(list(zip(X_test.columns,rgs.best_estimator_.coef_)))

,0,1
0,age,452.166524
1,male,-1884.889735
2,temperature,42.770997
3,relative_humidity,-4.775034
4,wind_speed,32.974044
5,precipitation,69.559653
6,precipitation_cover,3.730078
7,cloud_cover,-8.184404
8,overcast,0.000000
9,partially_cloudy,494.768063


In [56]:
train = nyc[nyc.index.isin(range(2017), level=0)]
test = nyc[nyc.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


rgs.fit(X_train, y_train)
model_scores(X_test, y_test, rgs)

R2: 0.8743049227803631
MAE: 571.2705548309202
MSE: 489261.3468431995
RMSE: 699.4721916153633


In [57]:
rgs.best_params_

{'alpha': 10, 'tol': 1}

In [58]:
pd.DataFrame(list(zip(X_test.columns,rgs.best_estimator_.coef_)))

,0,1
0,age,748.706657
1,male,-1647.392727
2,temperature,28.929294
3,relative_humidity,-0.003576
4,wind_speed,11.904176
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,5.372534
8,overcast,-187.571310
9,partially_cloudy,-172.408137


In [59]:
train = boston[boston.index.isin(range(2017), level=0)]
test = boston[boston.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear', 'overcast', 'partially_cloudy'])
X_test = test.drop(columns = ['time_seconds', 'clear', 'overcast', 'partially_cloudy'])

y_train = train['time_seconds']
y_test = test['time_seconds']


rgs.fit(X_train, y_train)
model_scores(X_test, y_test, rgs)

R2: 0.4178721813799452
MAE: 1289.690741432019
MSE: 2153689.4382674033
RMSE: 1467.5453786058554


In [60]:
pd.DataFrame(list(zip(X_test.columns,rgs.best_estimator_.coef_)))

,0,1
0,age,589.097417
1,male,-1237.069923
2,temperature,86.406290
3,relative_humidity,12.018486
4,wind_speed,8.963456
5,precipitation,71.070933
6,precipitation_cover,4.012326
7,cloud_cover,-13.474268
8,rain,90.037693
9,event,0.000000


In [61]:
train = berlin[berlin.index.isin(range(2017), level=0)]
test = berlin[berlin.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


rgs.fit(X_train, y_train)
model_scores(X_test, y_test, rgs)

R2: 0.6322299285866535
MAE: 794.726182859107
MSE: 816952.0918424213
RMSE: 903.8540213123032


In [62]:
pd.DataFrame(list(zip(X_test.columns,rgs.best_estimator_.coef_)))

,0,1
0,age,464.332046
1,male,-1341.315157
2,temperature,65.660304
3,relative_humidity,3.169582
4,wind_speed,-14.157234
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,-4.552160
8,overcast,326.408294
9,partially_cloudy,337.368708


In [63]:
train = chicago[chicago.index.isin(range(2017), level=0)]
test = chicago[chicago.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


rgs.fit(X_train, y_train)
model_scores(X_test, y_test, rgs)

R2: 0.7880189266044859
MAE: 625.1907916340756
MSE: 591502.5910986601
RMSE: 769.0920563226877


In [64]:
pd.DataFrame(list(zip(X_test.columns,rgs.best_estimator_.coef_)))

,0,1
0,age,444.285030
1,male,-1358.301966
2,temperature,45.888012
3,relative_humidity,-11.945177
4,wind_speed,-50.946818
5,precipitation,0.005821
6,precipitation_cover,5.821394
7,cloud_cover,10.504392
8,overcast,28.625771
9,partially_cloudy,-444.943478


In [65]:
train = combined[combined.index.isin(range(2017), level=0)]
test = combined[combined.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


rgs.fit(X_train, y_train)
model_scores(X_test, y_test, rgs)

R2: 0.5368886728819522
MAE: 1055.650498060206
MSE: 1689585.3218376744
RMSE: 1299.8404986142239


In [66]:
rgs.best_estimator_

Ridge(alpha=1, tol=1)

In [67]:
pd.DataFrame(list(zip(X_test.columns,rgs.best_estimator_.coef_)))

,0,1
0,age,546.986091
1,male,-1637.778966
2,temperature,23.841939
3,relative_humidity,-23.983239
4,wind_speed,-33.369796
5,precipitation,2571.454527
6,precipitation_cover,-16.546856
7,cloud_cover,4.021562
8,overcast,-650.499566
9,partially_cloudy,-20.537234


## Modeling Lasso

In [85]:
lparams = {
    'alpha': [.01, .1, 1, 10, 100, 1000],
    'tol': [1, .1, .01, .001, .0001, .00001, .000001, .0000001],
    'max_iter': [100_000]
}

lgs = GridSearchCV(
    estimator=Lasso(),
    param_grid=lparams
)

In [86]:
train = london[london.index.isin(range(2017), level=0)]
test = london[london.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


lgs.fit(X_train, y_train)
model_scores(X_test, y_test, lgs)

R2: 0.8493569006727154
MAE: 506.4387126106407
MSE: 362095.0542913358
RMSE: 601.7433458637792


In [87]:
pd.DataFrame(list(zip(X_test.columns,lgs.best_estimator_.coef_)))

,0,1
0,age,450.647935
1,male,-1875.789567
2,temperature,45.104321
3,relative_humidity,-7.532202
4,wind_speed,26.908565
5,precipitation,-0.000000
6,precipitation_cover,-6.160870
7,cloud_cover,0.424239
8,overcast,0.000000
9,partially_cloudy,50.430921


In [88]:
train = nyc[nyc.index.isin(range(2017), level=0)]
test = nyc[nyc.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


lgs.fit(X_train, y_train)
model_scores(X_test, y_test, lgs)

R2: 0.8691932934188619
MAE: 584.9532556587193
MSE: 509158.08998773247
RMSE: 713.5531444732989


In [89]:
lgs.best_params_

{'alpha': 10, 'max_iter': 100000, 'tol': 1}

In [90]:
pd.DataFrame(list(zip(X_test.columns,lgs.best_estimator_.coef_)))

,0,1
0,age,752.233869
1,male,-1865.978341
2,temperature,26.142319
3,relative_humidity,1.922232
4,wind_speed,13.219215
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,2.966547
8,overcast,-0.000000
9,partially_cloudy,-60.428015


In [91]:
train = boston[boston.index.isin(range(2017), level=0)]
test = boston[boston.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear', 'overcast', 'partially_cloudy'])
X_test = test.drop(columns = ['time_seconds', 'clear', 'overcast', 'partially_cloudy'])

y_train = train['time_seconds']
y_test = test['time_seconds']


lgs.fit(X_train, y_train)
model_scores(X_test, y_test, lgs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.228e+07, tolerance: 7.535e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.228e+07, tolerance: 7.535e+05
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.228e+07, toleranc

R2: 0.36044966105465814
MAE: 1400.5958818167537
MSE: 2366134.6635040683
RMSE: 1538.2245166113003


In [92]:
pd.DataFrame(list(zip(X_test.columns,lgs.best_estimator_.coef_)))

,0,1
0,age,591.975880
1,male,-1391.670545
2,temperature,86.380376
3,relative_humidity,8.064015
4,wind_speed,15.087688
5,precipitation,0.000000
6,precipitation_cover,1.282255
7,cloud_cover,-9.135620
8,rain,181.916244
9,event,0.000000


In [93]:
train = berlin[berlin.index.isin(range(2017), level=0)]
test = berlin[berlin.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


lgs.fit(X_train, y_train)
model_scores(X_test, y_test, lgs)

R2: 0.6589608344227431
MAE: 769.4512907622236
MSE: 757572.9548840689
RMSE: 870.3866697532017


In [94]:
pd.DataFrame(list(zip(X_test.columns,lgs.best_estimator_.coef_)))

,0,1
0,age,465.175697
1,male,-1416.500832
2,temperature,63.696596
3,relative_humidity,1.132600
4,wind_speed,-15.783847
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,-0.499123
8,overcast,0.000000
9,partially_cloudy,145.899696


In [95]:
train = chicago[chicago.index.isin(range(2017), level=0)]
test = chicago[chicago.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


lgs.fit(X_train, y_train)
model_scores(X_test, y_test, lgs)

R2: 0.8301187947562846
MAE: 555.6399519380373
MSE: 474028.98509309755
RMSE: 688.4976289669396


In [96]:
pd.DataFrame(list(zip(X_test.columns,lgs.best_estimator_.coef_)))

,0,1
0,age,445.268094
1,male,-1493.701748
2,temperature,45.974793
3,relative_humidity,-11.260369
4,wind_speed,-51.084860
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,11.137780
8,overcast,-0.000000
9,partially_cloudy,-499.682008


In [97]:
train = combined[combined.index.isin(range(2017), level=0)]
test = combined[combined.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


lgs.fit(X_train, y_train)
model_scores(X_test, y_test, lgs)

R2: 0.5456792012125993
MAE: 1041.158958860753
MSE: 1657514.5285554496
RMSE: 1287.4449613693976


In [98]:
lgs.best_estimator_

Lasso(alpha=0.1, max_iter=100000, tol=1)

In [99]:
pd.DataFrame(list(zip(X_test.columns,lgs.best_estimator_.coef_)))

,0,1
0,age,546.823145
1,male,-1637.678620
2,temperature,25.570423
3,relative_humidity,-24.174118
4,wind_speed,-33.054812
5,precipitation,2076.939515
6,precipitation_cover,-11.029852
7,cloud_cover,0.273125
8,overcast,-336.363789
9,partially_cloudy,160.853338


## Elastic Net

In [128]:
enparams = {
    'alpha': [.01, .1, 1, 10, 100, 1000],
    'max_iter': [10_000_000],
    'l1_ratio': np.linspace(0,1,100)
}

engs = GridSearchCV(
    estimator=ElasticNet(),
    param_grid=enparams
)

In [129]:
train = london[london.index.isin(range(2017), level=0)]
test = london[london.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


engs.fit(X_train, y_train)
model_scores(X_test, y_test, engs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.968e+07, tolerance: 4.917e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.230e+07, tolerance: 5.196e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

R2: 0.8332823748033237
MAE: 541.2377440321928
MSE: 400732.77711685584
RMSE: 633.0345781368154


In [130]:
engs.best_params_

{'alpha': 100, 'l1_ratio': 1.0, 'max_iter': 10000000}

In [131]:
pd.DataFrame(list(zip(X_test.columns,engs.best_estimator_.coef_)))

,0,1
0,age,433.871770
1,male,-1515.789567
2,temperature,38.115665
3,relative_humidity,-5.234150
4,wind_speed,25.627689
5,precipitation,-0.000000
6,precipitation_cover,-12.290341
7,cloud_cover,-0.000000
8,overcast,0.000000
9,partially_cloudy,0.000000


In [132]:
train = nyc[nyc.index.isin(range(2017), level=0)]
test = nyc[nyc.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


engs.fit(X_train, y_train)
model_scores(X_test, y_test, engs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.623e+07, tolerance: 8.368e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.539e+07, tolerance: 8.785e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

R2: 0.8772893572531737
MAE: 571.4053671524455
MSE: 477644.59571792564
RMSE: 691.1183659243369


In [133]:
engs.best_params_

{'alpha': 0.1, 'l1_ratio': 0.7676767676767677, 'max_iter': 10000000}

In [134]:
pd.DataFrame(list(zip(X_test.columns,engs.best_estimator_.coef_)))

,0,1
0,age,750.914270
1,male,-1743.383536
2,temperature,30.123849
3,relative_humidity,-0.831466
4,wind_speed,11.050070
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,6.527796
8,overcast,-280.802641
9,partially_cloudy,-226.177620


In [135]:
train = boston[boston.index.isin(range(2017), level=0)]
test = boston[boston.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear', 'overcast', 'partially_cloudy'])
X_test = test.drop(columns = ['time_seconds', 'clear', 'overcast', 'partially_cloudy'])

y_train = train['time_seconds']
y_test = test['time_seconds']


engs.fit(X_train, y_train)
model_scores(X_test, y_test, engs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.003e+07, tolerance: 7.535e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.066e+07, tolerance: 6.563e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

R2: 0.40224682180474836
MAE: 1297.471038186949
MSE: 2211498.3434765805
RMSE: 1487.1107367901627


In [136]:
engs.best_params_

{'alpha': 100, 'l1_ratio': 1.0, 'max_iter': 10000000}

In [137]:
pd.DataFrame(list(zip(X_test.columns,engs.best_estimator_.coef_)))

,0,1
0,age,574.176998
1,male,-1030.933494
2,temperature,85.986731
3,relative_humidity,11.581374
4,wind_speed,5.206532
5,precipitation,0.000000
6,precipitation_cover,5.718157
7,cloud_cover,-12.764714
8,rain,0.000000
9,event,0.000000


In [138]:
train = berlin[berlin.index.isin(range(2017), level=0)]
test = berlin[berlin.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


engs.fit(X_train, y_train)
model_scores(X_test, y_test, engs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.827e+07, tolerance: 8.361e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.465e+08, tolerance: 9.986e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

R2: 0.6353785852966253
MAE: 761.3116729376164
MSE: 809957.7715166273
RMSE: 899.9765394256826


In [139]:
engs.best_params_

{'alpha': 100, 'l1_ratio': 1.0, 'max_iter': 10000000}

In [140]:
pd.DataFrame(list(zip(X_test.columns,engs.best_estimator_.coef_)))

,0,1
0,age,444.539436
1,male,-1053.900873
2,temperature,57.206784
3,relative_humidity,-0.000000
4,wind_speed,-14.706931
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,-0.121331
8,overcast,-0.000000
9,partially_cloudy,0.000000


In [141]:
train = chicago[chicago.index.isin(range(2017), level=0)]
test = chicago[chicago.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


engs.fit(X_train, y_train)
model_scores(X_test, y_test, engs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.792e+07, tolerance: 5.690e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.146e+07, tolerance: 6.575e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

R2: 0.8301541678680803
MAE: 555.5786970598895
MSE: 473930.28152986313
RMSE: 688.4259448407382


In [142]:
engs.best_params_

{'alpha': 10, 'l1_ratio': 1.0, 'max_iter': 10000000}

In [143]:
pd.DataFrame(list(zip(X_test.columns,engs.best_estimator_.coef_)))

,0,1
0,age,445.266579
1,male,-1493.647453
2,temperature,45.998546
3,relative_humidity,-11.285113
4,wind_speed,-51.055517
5,precipitation,0.000000
6,precipitation_cover,0.000000
7,cloud_cover,11.149686
8,overcast,-0.000000
9,partially_cloudy,-500.050616


In [125]:
train = combined[combined.index.isin(range(2017), level=0)]
test = combined[combined.index.isin([2017, 2018], level=0)]

X_train = train.drop(columns = ['time_seconds', 'clear'])
X_test = test.drop(columns = ['time_seconds', 'clear'])

y_train = train['time_seconds']
y_test = test['time_seconds']


engs.fit(X_train, y_train)
model_scores(X_test, y_test, engs)

C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.090e+09, tolerance: 1.934e+06 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\David\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.152e+09, tolerance: 2.003e+06 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

R2: 0.5428052616946761
MAE: 1033.8972747621153
MSE: 1667999.6230478466
RMSE: 1291.510597342448


In [126]:
engs.best_estimator_

ElasticNet(alpha=0.01, l1_ratio=0.9696969696969697, max_iter=1000000)

In [127]:
pd.DataFrame(list(zip(X_test.columns,engs.best_estimator_.coef_)))

,0,1
0,age,546.967921
1,male,-1636.759598
2,temperature,23.927350
3,relative_humidity,-23.866213
4,wind_speed,-32.707802
5,precipitation,2082.901731
6,precipitation_cover,-15.715567
7,cloud_cover,3.879328
8,overcast,-641.804866
9,partially_cloudy,-13.780661
